In [1]:
import os
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wget

In [2]:
def load_data(filePath):
    df = pd.read_csv(filePath)
    return df

## Load CSV-data into DataFrames

In [3]:
# Prepped Time Series Data with datapoints for each date, for each country
PREPPED_DATA = "output/COVID9-cleaned_03-05-2020_21:13.csv"
complete_df = load_data(PREPPED_DATA)

# Prepped data per country
COUNTRY_DATA = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv"
country_df = pd.read_csv(COUNTRY_DATA)
country_df.rename(columns={'Country_Region':'Country', 'Last_Update':'Date', 'Long_':'Long'}, inplace=True)
country_df = country_df[['Country', 'Date', 'Lat', 'Long', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'UID', 'ISO3']]


## Prepare DataFrames for plotting with Plotly

In [4]:
top_5 = ['US', 'Spain', 'Italy', 'UK', 'France']
top_10 = ['US', 'Spain', 'Italy', 'UK', 'France', 'Germany', 'Turkey', 'Russia', 'Brazil', 'Iran'] 

# Create a new DataFrame for the chosen countries
top_countries_df = complete_df[pd.DataFrame(complete_df['Country'].tolist()).isin(top_10).any(1)]
# Sort out data from before 100 Confirmed cases
top_countries_100_df = top_countries_df[top_countries_df['Confirmed'] >= 100] 

norge = complete_df[complete_df['Country'] == 'Norway']
norge = norge[norge['Confirmed'] >= 100]

In [5]:
import plotly.express as px
n=10
sorted_country_df = country_df.sort_values('Confirmed', ascending=False)

fig = px.scatter(sorted_country_df.head(n), x="Country", y="Deaths", size="Confirmed", color="Country",hover_name="Country", size_max=80)

fig.update_layout(
title=f"Top {n} worst hit countries",
xaxis_title="Countries",
yaxis_title="Deaths",
)

fig.update_layout(
    autosize=False,
    width=800,
    height=450,
    plot_bgcolor='white')


fig.show()

In [6]:
import plotly.express as px
px.set_mapbox_access_token(open(".mapbox_token").read())


df = country_df
fig = px.scatter_mapbox(
    df, lat="Lat", lon="Long",
    color="Deaths", size="Confirmed",
    size_max=40,
    zoom=0,
    color_continuous_scale="viridis"
    )
# "open-street-map", "carto-positron", "carto-darkmatter", "stamen-terrain", "stamen-toner" or "stamen-watercolor" yeild maps composed of raster tiles from various public tile servers which do not  require signups or access tokens

# "basic", "streets", "outdoors", "light", "dark", "satellite", or "satellite-streets" yeild maps composed of vector tiles from the Mapbox service, and do require a Mapbox Access Token or an on-premise Mapbox installation.

# https://plotly.com/python/builtin-colorscales/
# https://plotly.com/python/discrete-color/

# https://plotly.com/python/mapbox-layers/


fig.update_layout(
    autosize=False,
    width=800,
    height=450,
    #mapbox_style='carto-darkmatter'
)

fig.show()

In [7]:
import plotly.express as px
df = country_df.sort_values('Deaths', ascending=False).head(5)
fig = px.scatter_3d(
    df, x="Country", y="Confirmed", z="Deaths",
    color="Recovered", size="Deaths", size_max=125, hover_name="Country",
    color_continuous_scale="Viridis")

fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    template='plotly_white'
)


fig.show()

In [8]:
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

import plotly.graph_objects as go

def plot_cases_of_a_country(country):
    labels = ['confirmed', 'deaths']
    colors = ['blue', 'red']
    mode_size = [6, 8]
    line_size = [4, 5]
    
    df_list = [confirmed_df, death_df]
    
    fig = go.Figure()
    
    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,4:]),axis = 0)
            
        else:    
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df[df['Country/Region'] == country].iloc[:,20:]),axis = 0)
            
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
        text = "Total " + str(labels[i]) +": "+ str(y_data[-1])
        ))
    
    fig.update_layout(
        title="COVID 19 cases of " + country,
        xaxis_title='Date',
        yaxis_title='No. of Confirmed Cases',
        margin=dict(l=20, r=20, t=40, b=20),
        plot_bgcolor='white'
    )
    
    fig.update_yaxes(type="linear")
    fig.show()

In [9]:
plot_cases_of_a_country("Norway")

In [10]:
sorted_country_df = country_df.sort_values('Confirmed', ascending= False)

fig = px.bar(sorted_country_df.head(10), x='Country', y='Active')

fig.update_layout(
    autosize=False,
    width=800,
    height=450,
    plot_bgcolor='white')

fig.update_yaxes(type="log")
fig.show()

In [48]:

fig = px.line(top_countries_100_df, x='Date', y='Confirmed', color='Country', line_group='Country', title='Confirmed cases comparison')
fig.update_layout(
        xaxis=dict(
            showline=True,
            showgrid=False,
            showticklabels=True,
            linecolor='rgb(204, 204, 204)',
            linewidth=2,
            ticks='outside',
            tickfont=dict(
                family='Arial',
                size=14)
        ))

fig.update_yaxes(type="log")
#fig.update_layout(xaxis_rangeslider_visible=True)

fig.update_layout(
    autosize=False,
    width=800,
    height=800,
    plot_bgcolor='white')


#fig.write_image("images/CONFIRMED_CASES_NORWAY.pdf")
fig.show()


In [39]:
import plotly.graph_objects as go

import pandas as pd

# Get Data: this ex will only use part of it (i.e. rows 750-1500)
df = top_countries_100_df

start, end = 0, -1

fig = go.Figure(data=go.Scatter3d(
    x=df['Date'],
    y=df['Country'],
    z=df['New Cases'],
    text=df['Country'],
    mode='markers',
    marker=dict(
        sizemode='diameter',
        sizeref=800,
        size=df['Deaths'],
        color = df['New Cases'],
        colorscale = 'Viridis',
        colorbar_title = 'Deaths',
        line_color='rgb(128, 128, 128)'
        )
    )
)


fig.update_layout(
    title = 'New Cases Over Time',
    width=800, height=800, 
    scene_aspectmode='manual',
    scene_aspectratio=dict(x=3, y=1, z=2),
    scene = dict(
                xaxis=dict(title='Date'),
                yaxis=dict(title='Country'),
                zaxis=dict(title='Daily New Cases')
                )
)

fig.update_layout(scene_zaxis_type="log")

fig.show()

In [19]:
import plotly.express as px

df = top_countries_100_df

fig = px.scatter(df, x="Date", y="Country", size="New Cases", color="Country", size_max=40)

fig.update_layout(
title=f"Top 10 worst hit countries",
xaxis_title="Date",
yaxis_title="Conutry",
)

fig.update_layout(
    autosize=False,
    width=800,
    height=800,
    plot_bgcolor='white'
)

fig.update_layout(scene_yaxis_type="log")

fig.show()

In [35]:
import plotly.express as px
df = top_countries_100_df
fig = px.treemap(data_frame=df, path=['Country'], values='Confirmed', color_discrete_sequence=px.colors.qualitative.G10
)
fig.show()